In [1]:
import graphlab

In [2]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')

[INFO] This non-commercial license of GraphLab Create is assigned to jessherlock@sjtu.edu.cnand will expire on October 13, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-26813 - Server binary: /Library/Python/2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1445999509.log
[INFO] GraphLab Server Version: 1.6.1


# Exploring the image data

In [3]:
graphlab.canvas.set_target('ipynb')

In [4]:
image_train['image'].show()

In [5]:
image_train

Columns:
	id	int
	image	Image
	label	str
	deep_features	array
	image_array	array

Rows: 2005

Data:
+-----+----------------------+------------+-------------------------------+
|  id |        image         |   label    |         deep_features         |
+-----+----------------------+------------+-------------------------------+
|  24 | Height: 32 Width: 32 |    bird    | [0.242871761322, 1.0954537... |
|  33 | Height: 32 Width: 32 |    cat     | [0.525087952614, 0.0, 0.0,... |
|  36 | Height: 32 Width: 32 |    cat     | [0.566015958786, 0.0, 0.0,... |
|  70 | Height: 32 Width: 32 |    dog     | [1.12979578972, 0.0, 0.0, ... |
|  90 | Height: 32 Width: 32 |    bird    | [1.71786928177, 0.0, 0.0, ... |
|  97 | Height: 32 Width: 32 | automobile | [1.57818555832, 0.0, 0.0, ... |
| 107 | Height: 32 Width: 32 |    dog     | [0.0, 0.0, 0.220677852631,... |
| 121 | Height: 32 Width: 32 |    bird    | [0.0, 0.23753464222, 0.0, ... |
| 136 | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.0, 0.0, ... |
| 138 | Height: 32 Width: 32 |    bird    | [0.658935725689, 0.0, 0.0,... |
+-----+----------------------+------------+-------------------------------+
+-------------------------------+
|          image_array          |
+-------------------------------+
| [73.0, 77.0, 58.0, 71.0, 6... |
| [7.0, 5.0, 8.0, 7.0, 5.0, ... |
| [169.0, 122.0, 65.0, 131.0... |
| [154.0, 179.0, 152.0, 159.... |
| [216.0, 195.0, 180.0, 201.... |
| [33.0, 44.0, 27.0, 29.0, 4... |
| [97.0, 51.0, 31.0, 104.0, ... |
| [93.0, 96.0, 88.0, 102.0, ... |
| [35.0, 59.0, 53.0, 36.0, 5... |
| [205.0, 193.0, 195.0, 200.... |
+-------------------------------+
[2005 rows x 5 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

# Train a classifier on the raw image pixels

In [6]:
raw_pixel_model = graphlab.logistic_classifier.create(image_train,
                                                     target = 'label',
                                                     features=['image_array'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 1907
PROGRESS: Number of classes           : 4
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 3072
PROGRESS: Number of coefficients    : 9219
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 6        | 0.000015  | 5.037985     | 0.352386          | 0.285714            |
PROGR

In [7]:
image_test[0:3]['image'].show()

In [10]:
image_test[0:3]['label']

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

In [11]:
raw_pixel_model.predict(image_test[0:3])

dtype: str
Rows: 3
['bird', 'cat', 'bird']

# Evaluate raw pixel model on test data

In [12]:
raw_pixel_model.evaluate(image_test)

{'accuracy': 0.4765, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     dog      |    automobile   |  117  |
 |     cat      |       dog       |  308  |
 |     dog      |       dog       |  432  |
 |     dog      |       cat       |  223  |
 |  automobile  |       bird      |  104  |
 |     bird     |       bird      |  512  |
 |     bird     |       dog       |  192  |
 |     cat      |       cat       |  321  |
 |     dog      |       bird      |  228  |
 |     cat      |    automobile   |  176  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.}

# Can we improve the model using deep features

In [13]:
len(image_train)

2005

In [ ]:
deep_learning_model = graphlab.load_model('imagenet_model')
image_train['deep_features'] = deep_learning_model.extract_features(image_train)

# Given the deep features, let's train a classifier

In [16]:
deep_features_model = graphlab.logistic_classifier.create(image_train,
                                                          target='label',
                                                         features=['deep_features'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: WARNING: Detected extremely low variance for feature(s) 'deep_features' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.
PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 1900
PROGRESS: Number of classes           : 4
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 4096
PROGRESS: Number of coefficients    : 12291
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteratio

In [17]:
deep_features_model.predict(image_test[0:3])

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

In [19]:
image_test[0:3]['label']

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

In [20]:
deep_features_model.evaluate(image_test)

{'accuracy': 0.78425, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |  automobile  |       cat       |   10  |
 |     dog      |       cat       |  202  |
 |  automobile  |       dog       |   5   |
 |     cat      |       bird      |  103  |
 |     bird     |       dog       |   51  |
 |     dog      |       bird      |   60  |
 |     cat      |    automobile   |   43  |
 |     bird     |       cat       |  100  |
 |     dog      |    automobile   |   18  |
 |     dog      |       dog       |  720  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.}

In [21]:
knn_model = graphlab.nearest_neighbors.create(image_train,
                                             features=['deep_features'],
                                             label = 'id')

PROGRESS: Starting brute force nearest neighbors model training.


In [22]:
cat = image_train[18:19]

In [23]:
cat['image'].show()

In [24]:
knn_model.query(cat)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 24.093ms     |
PROGRESS: | Done         |         | 100         | 348.301ms    |
PROGRESS: +--------------+---------+-------------+--------------+


Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |       384       |      0.0      |  1   |
|      0      |       6910      | 36.9403137951 |  2   |
|      0      |      39777      | 38.4634888975 |  3   |
|      0      |      36870      | 39.7559623119 |  4   |
|      0      |      41734      | 39.7866014148 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

In [26]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [60]:
image_train[image_train['id']==6910]['image'].show()

In [27]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 25.276ms     |
PROGRESS: | Done         |         | 100         | 350.406ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [28]:
cat_neighbors['image'].show()

In [29]:
car = image_train[8:9]

In [30]:
car['image'].show()

In [31]:
get_images_from_ids(knn_model.query(car))['image'].show()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 23.365ms     |
PROGRESS: | Done         |         | 100         | 354.235ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [32]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [33]:
show_neighbors(8)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 26.117ms     |
PROGRESS: | Done         |         | 100         | 343.186ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [34]:
show_neighbors(26)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 23.844ms     |
PROGRESS: | Done         |         | 100         | 331.352ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [35]:
show_neighbors(1222)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 26.214ms     |
PROGRESS: | Done         |         | 100         | 328.533ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [36]:
show_neighbors(2000)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 41.142ms     |
PROGRESS: | Done         |         | 100         | 335.075ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [37]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


## Answer 1: least common category in training data is `bird`

In [51]:
dogs = image_train[image_train['label']=='dog']
cats = image_train[image_train['label']=='cat']
automobiles = image_train[image_train['label']=='automobile']
birds = image_train[image_train['label']=='bird']

In [52]:
dogs_model = graphlab.nearest_neighbors.create(dogs,
                                                  features=['deep_features'],
                                                  label='id')
cats_model = graphlab.nearest_neighbors.create(cats,
                                                  features=['deep_features'],
                                                  label='id')
automobiles_model = graphlab.nearest_neighbors.create(automobiles,
                                                  features=['deep_features'],
                                                  label='id')
birds_model = graphlab.nearest_neighbors.create(birds,
                                                features=['deep_features'],
                                                label='id')

PROGRESS: Starting brute force nearest neighbors model training.
PROGRESS: Starting brute force nearest neighbors model training.
PROGRESS: Starting brute force nearest neighbors model training.
PROGRESS: Starting brute force nearest neighbors model training.


In [57]:
image_test[0:1]['image'].show()

In [100]:
cats_model.query(image_test[0:1])

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 24.865ms     |
PROGRESS: | Done         |         | 100         | 128.73ms     |
PROGRESS: +--------------+---------+-------------+--------------+


Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |      16289      |  34.623719208 |  1   |
|      0      |      45646      | 36.0068799284 |  2   |
|      0      |      32139      | 36.5200813436 |  3   |
|      0      |      25713      | 36.7548502521 |  4   |
|      0      |       331       | 36.8731228168 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

In [101]:
image_train[image_train['id']==16289]['image'].show()

In [102]:
dogs_model.query(image_test[0:1])

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 54.135ms     |
PROGRESS: | Done         |         | 100         | 153.472ms    |
PROGRESS: +--------------+---------+-------------+--------------+


Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |      16976      | 37.4642628784 |  1   |
|      0      |      13387      | 37.5666832169 |  2   |
|      0      |      35867      | 37.6047267079 |  3   |
|      0      |      44603      | 37.7065585153 |  4   |
|      0      |       6094      | 38.5113254907 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

In [107]:
image_train[image_train['id']==16976]['image'].show()

In [62]:
cats_model.query(image_test[0:1])['distance'].mean()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 29.862ms     |
PROGRESS: | Done         |         | 100         | 147.822ms    |
PROGRESS: +--------------+---------+-------------+--------------+


36.15573070978294

In [63]:
dogs_model.query(image_test[0:1])['distance'].mean()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 34.862ms     |
PROGRESS: | Done         |         | 100         | 151.897ms    |
PROGRESS: +--------------+---------+-------------+--------------+


37.77071136184156

# Computing nearest neighbors accuracy using SFrame operations

In [66]:
image_test_dog = image_test[image_test['label']=='dog']
image_test_cat = image_test[image_test['label']=='cat']
image_test_automobile = image_test[image_test['label']=='automobile']
image_test_bird = image_test[image_test['label']=='bird']

In [72]:
dog_cat_neighbors = cats_model.query(image_test_dog, k=1)
dog_dog_neighbors = dogs_model.query(image_test_dog, k=1)
dog_automobile_neighbors = automobiles_model.query(image_test_dog, k=1)
dog_bird_neighbors = birds_model.query(image_test_dog, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 127000  | 24.9509     | 580.32ms     |
PROGRESS: | Done         | 509000  | 100         | 701.882ms    |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 127000  | 24.9509     | 591.62ms     |
PROGRESS: 

In [69]:
dog_cat_neighbors

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 1000

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |        33       | 36.4196077068 |  1   |
|      1      |      30606      | 38.8353268874 |  1   |
|      2      |       5545      | 36.9763410854 |  1   |
|      3      |      19631      | 34.5750072914 |  1   |
|      4      |       7493      |  34.778824791 |  1   |
|      5      |      47044      | 35.1171578292 |  1   |
|      6      |      13918      | 40.6095830913 |  1   |
|      7      |      10981      | 39.9036867306 |  1   |
|      8      |      45456      | 38.0674700168 |  1   |
|      9      |      44673      | 42.7258732951 |  1   |
+-------------+-----------------+---------------+------+
[1000 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

## Create an SFrame with the distances from `dog` test examples to the respective nearest neighbors in each class in the training data

In [75]:
new_sf = graphlab.SFrame({'dog-dog':dog_dog_neighbors['distance'],
                         'dog-cat':dog_cat_neighbors['distance'],
                         'dog-automobile':dog_automobile_neighbors['distance'],
                         'dog-bird':dog_bird_neighbors['distance']})

In [76]:
new_sf

Columns:
	dog-automobile	float
	dog-bird	float
	dog-cat	float
	dog-dog	float

Rows: 1000

Data:
+----------------+---------------+---------------+---------------+
| dog-automobile |    dog-bird   |    dog-cat    |    dog-dog    |
+----------------+---------------+---------------+---------------+
| 41.9579761457  | 41.7538647304 | 36.4196077068 | 33.4773590373 |
| 46.0021331807  | 41.3382958925 | 38.8353268874 | 32.8458495684 |
| 42.9462290692  | 38.6157590853 | 36.9763410854 | 35.0397073189 |
| 41.6866060048  | 37.0892269954 | 34.5750072914 | 33.9010327697 |
| 39.2269664935  |  38.272288694 |  34.778824791 | 37.4849250909 |
| 40.5845117698  | 39.1462089236 | 35.1171578292 |  34.945165344 |
| 45.1067352961  |  40.523040106 | 40.6095830913 | 39.0957278345 |
| 41.3221140974  | 38.1947918393 | 39.9036867306 | 37.7696131032 |
| 41.8244654995  | 40.1567131661 | 38.0674700168 | 35.1089144603 |
| 45.4976929401  | 45.5597962603 | 42.7258732951 | 43.2422832585 |
+----------------+---------------+---------------+---------------+
[1000 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [83]:
new_sf[0].values()

[36.41960770675437, 33.47735903726335, 41.753864730351246, 41.957976145712024]

In [85]:
min(new_sf[0].values())==new_sf[0]['dog-dog']

True

In [87]:
def is_dog_correct(row):
    if row['dog-dog'] == min(row.values()): return 1
    else: return 0

In [90]:
dog_test_predictions =new_sf.apply(is_dog_correct)

In [91]:
dog_test_predictions.sum()

678

In [92]:
cat_cat_neighbors = cats_model.query(image_test_cat, k=1)
cat_dog_neighbors = dogs_model.query(image_test_cat, k=1)
cat_automobile_neighbors = automobiles_model.query(image_test_cat, k=1)
cat_bird_neighbors = birds_model.query(image_test_cat, k=1)
cat_distances = graphlab.SFrame({'cat-cat':cat_cat_neighbors['distance'],
                                'cat-dog':cat_dog_neighbors['distance'],
                                'cat-automobile':cat_automobile_neighbors['distance'],
                                'cat-birds':cat_bird_neighbors['distance']})


PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 127000  | 24.9509     | 591.888ms    |
PROGRESS: | Done         | 509000  | 100         | 669.691ms    |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 127000  | 24.9509     | 563.866ms    |
PROGRESS: 

In [93]:
def is_cat_correct(row):
    if row['cat-cat'] == min(row.values()): return 1
    else : return 0

In [94]:
cat_distances.apply(is_cat_correct).sum()

548